In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [6]:
loss_thresh = 0.1
label = "KG_stan"

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [7]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z)
            a = z1 + self.beta[:,i]*z*z1            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [9]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [10]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
#     xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,seed*123)
#     xt_coll = torch.from_numpy(xt_coll).float().to(device)
#     xt_BC = torch.from_numpy(xt_BC).float().to(device)
#     y_BC = torch.from_numpy(y_BC).float().to(device)

#     f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*9)
    start_time = time.time() 
    thresh_flag = 0

    xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_BC = torch.from_numpy(xt_BC).float().to(device)
    y_BC = torch.from_numpy(y_BC).float().to(device)

    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    

    for i in range(max_iter):
        train_step(xt_BC, y_BC, xt_coll,f_hat,i)
        loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1          
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])   
        
      
         

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
max_reps = 10 #10
max_iter = 100 #200

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

beta_init = 0

N_I = 200  #Total number of data points for 'y'
N_B = 400
N_f = 10000 #Total number of collocation points

for reps in range(max_reps):
    print(label)
    print(reps)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []
    beta_val = []

    torch.manual_seed(reps*36)

    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    #layers = np.array([2,50,50,50,50,50,50,50,1])

    PINN = Sequentialmodel(layers,beta_init)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




    nan_flag = train_model(max_iter,reps)



    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    beta_full.append(beta_val)


  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

KG_stan
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.162746 Test MSE 8.522083219762841 Test RE 1.3953428962386416
1 Train Loss 40.840633 Test MSE 7.151209556321126 Test RE 1.2781970550170474
2 Train Loss 34.543537 Test MSE 7.194036724115823 Test RE 1.2820187752975112
3 Train Loss 30.870075 Test MSE 6.403641979394024 Test RE 1

92 Train Loss 0.003122013 Test MSE 0.0010090875177041002 Test RE 0.015183520400129628
93 Train Loss 0.0029801542 Test MSE 0.0009425234125492712 Test RE 0.014674189858306593
94 Train Loss 0.0028886308 Test MSE 0.0009176611239968836 Test RE 0.014479355364123161
95 Train Loss 0.0027903053 Test MSE 0.0008356369210295396 Test RE 0.013817099033298062
96 Train Loss 0.0026477717 Test MSE 0.0007831095828819455 Test RE 0.013375786543755864
97 Train Loss 0.002536909 Test MSE 0.000749723560035164 Test RE 0.013087558611876179
98 Train Loss 0.002446138 Test MSE 0.0007921571236676511 Test RE 0.013452832234956403
99 Train Loss 0.002387318 Test MSE 0.0007612059302001305 Test RE 0.013187398857835885
Training time: 125.27
KG_stan
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    

88 Train Loss 0.92505103 Test MSE 2.5376675040120777 Test RE 0.7614219975529134
89 Train Loss 0.9190612 Test MSE 2.524477449730249 Test RE 0.7594405949673817
90 Train Loss 0.90900695 Test MSE 2.54416634910806 Test RE 0.7623963567876346
91 Train Loss 0.8984829 Test MSE 2.576823311875316 Test RE 0.7672738212273884
92 Train Loss 0.8785628 Test MSE 2.611627342912597 Test RE 0.7724380587295463
93 Train Loss 0.8633232 Test MSE 2.630220179597373 Test RE 0.7751827734283536
94 Train Loss 0.8479759 Test MSE 2.6483609089230296 Test RE 0.7778514125489294
95 Train Loss 0.827513 Test MSE 2.6293745826915194 Test RE 0.7750581555632328
96 Train Loss 0.81040484 Test MSE 2.632493907706992 Test RE 0.7755177594733205
97 Train Loss 0.791877 Test MSE 2.6591238698641475 Test RE 0.7794304072451891
98 Train Loss 0.7792686 Test MSE 2.698204568427062 Test RE 0.7851370952999489
99 Train Loss 0.76317286 Test MSE 2.743414453586497 Test RE 0.7916874700548301
Training time: 126.07
KG_stan
2
Sequentialmodel(
  (activat

83 Train Loss 0.012776345 Test MSE 0.0019166739973927821 Test RE 0.020925805569664092
84 Train Loss 0.012302169 Test MSE 0.0017926497904943838 Test RE 0.02023744994702158
85 Train Loss 0.011564048 Test MSE 0.0015469002893601948 Test RE 0.01879919360972873
86 Train Loss 0.010920439 Test MSE 0.0016773507135509989 Test RE 0.019575821629438116
87 Train Loss 0.010017211 Test MSE 0.001814417532729782 Test RE 0.020359948596713584
88 Train Loss 0.009170726 Test MSE 0.001737576148314772 Test RE 0.019924158294386953
89 Train Loss 0.00858777 Test MSE 0.0016045092973222257 Test RE 0.019146049581945643
90 Train Loss 0.007855144 Test MSE 0.0015715888763884452 Test RE 0.01894861768624331
91 Train Loss 0.007647471 Test MSE 0.001418295956967459 Test RE 0.018000786988162
92 Train Loss 0.007553477 Test MSE 0.0014382549067737118 Test RE 0.018127002408498294
93 Train Loss 0.007309642 Test MSE 0.0013397393486577341 Test RE 0.017495172009849173
94 Train Loss 0.00715579 Test MSE 0.001347174050429935 Test RE 0

78 Train Loss 0.62000203 Test MSE 2.7533247802775582 Test RE 0.7931161292567335
79 Train Loss 0.61170995 Test MSE 2.7473516862227165 Test RE 0.7922553645949862
80 Train Loss 0.60280335 Test MSE 2.758787192416813 Test RE 0.7939024840976807
81 Train Loss 0.59593105 Test MSE 2.7803934401347505 Test RE 0.7970052604696576
82 Train Loss 0.58684796 Test MSE 2.7984134958002254 Test RE 0.7995838314514638
83 Train Loss 0.5796224 Test MSE 2.8152194482360424 Test RE 0.8019811992797016
84 Train Loss 0.5727607 Test MSE 2.842302614455438 Test RE 0.8058296028914121
85 Train Loss 0.5654696 Test MSE 2.838412133877937 Test RE 0.8052779132262579
86 Train Loss 0.55931616 Test MSE 2.847114255483917 Test RE 0.8065113957416705
87 Train Loss 0.55144405 Test MSE 2.863507466777003 Test RE 0.8088299424525177
88 Train Loss 0.54394925 Test MSE 2.902666640184682 Test RE 0.8143416378240563
89 Train Loss 0.5388553 Test MSE 2.922868433288094 Test RE 0.8171705256297581
90 Train Loss 0.53076416 Test MSE 2.942357580769233

...

64 Train Loss 0.019730182 Test MSE 0.00710995789084299 Test RE 0.04030338976912711
65 Train Loss 0.018282093 Test MSE 0.0067219857349250225 Test RE 0.03918833875238671
66 Train Loss 0.016354254 Test MSE 0.006265389449282418 Test RE 0.03783398575433025
67 Train Loss 0.01463763 Test MSE 0.0059584199619066094 Test RE 0.03689551827067784
68 Train Loss 0.013594089 Test MSE 0.005190394077264918 Test RE 0.034435645364866284
69 Train Loss 0.012493033 Test MSE 0.004197800051135859 Test RE 0.030968411884487106
70 Train Loss 0.011528874 Test MSE 0.0035950690234649424 Test RE 0.028659042519128455
71 Train Loss 0.010350712 Test MSE 0.004290678343399841 Test RE 0.031309132851535824
72 Train Loss 0.0092675 Test MSE 0.0036762222861260463 Test RE 0.028980704726252995
73 Train Loss 0.008451585 Test MSE 0.0028425784630420996 Test RE 0.02548380603692025
74 Train Loss 0.007730039 Test MSE 0.002389183325569727 Test RE 0.023363225341160056
75 Train Loss 0.007295138 Test MSE 0.0022224513842829976 Test RE 0.02

57 Train Loss 0.61078215 Test MSE 3.0167711422524732 Test RE 0.8301933357180579
58 Train Loss 0.5971455 Test MSE 3.0132374384537526 Test RE 0.8297069685399902
59 Train Loss 0.5867683 Test MSE 3.009315245633931 Test RE 0.8291667969712835
60 Train Loss 0.5786018 Test MSE 3.0254716114324074 Test RE 0.8313896265189943
61 Train Loss 0.5651484 Test MSE 3.0477265605007946 Test RE 0.834441817254833
62 Train Loss 0.5559639 Test MSE 3.0636847005194126 Test RE 0.836623567075065
63 Train Loss 0.54861784 Test MSE 3.0859726795613467 Test RE 0.8396612259814482
64 Train Loss 0.5380481 Test MSE 3.1101798722628162 Test RE 0.8429480561729955
65 Train Loss 0.53154933 Test MSE 3.1174229743053465 Test RE 0.8439290296951621
66 Train Loss 0.52429485 Test MSE 3.1217519995332546 Test RE 0.844514789575266
67 Train Loss 0.5179622 Test MSE 3.16613690726792 Test RE 0.8504972338069687
68 Train Loss 0.51169264 Test MSE 3.1614880807811603 Test RE 0.8498726135532951
69 Train Loss 0.50655144 Test MSE 3.15209183517053 Te

52 Train Loss 0.021409215 Test MSE 0.0026786038857944667 Test RE 0.02473787026263652
53 Train Loss 0.020062305 Test MSE 0.0026283440902411075 Test RE 0.02450468761228015
54 Train Loss 0.019005364 Test MSE 0.0019011591616242506 Test RE 0.020840939777601344
55 Train Loss 0.016946737 Test MSE 0.0017810513287692205 Test RE 0.02017187545819061
56 Train Loss 0.016248101 Test MSE 0.0015450067778606817 Test RE 0.018787684338791386
57 Train Loss 0.014970676 Test MSE 0.0013600527085162813 Test RE 0.017627305449450608
58 Train Loss 0.013138995 Test MSE 0.0012435510994425315 Test RE 0.016855431624979655
59 Train Loss 0.012793303 Test MSE 0.0013212147495293924 Test RE 0.017373797966295824
60 Train Loss 0.011892842 Test MSE 0.0010440688379475796 Test RE 0.015444456404190213
61 Train Loss 0.011279946 Test MSE 0.0010574958375493624 Test RE 0.015543449036915584
62 Train Loss 0.011073942 Test MSE 0.000983795634607092 Test RE 0.014992032183470535
63 Train Loss 0.01016727 Test MSE 0.0008286843218847717 Te

43 Train Loss 5.2730303 Test MSE 2.025720853934047 Test RE 0.6802959097287542
44 Train Loss 5.244485 Test MSE 2.0345682608293476 Test RE 0.6817798992758671
45 Train Loss 5.2247634 Test MSE 2.025162704946145 Test RE 0.6802021819499277
46 Train Loss 5.205559 Test MSE 2.016794018968576 Test RE 0.678795309460291
47 Train Loss 5.1433806 Test MSE 2.024636083714868 Test RE 0.6801137366611124
48 Train Loss 5.0907273 Test MSE 1.9997402865872345 Test RE 0.6759193169928833
49 Train Loss 5.0320263 Test MSE 1.9937908220971305 Test RE 0.6749130979694894
50 Train Loss 4.966297 Test MSE 2.0106135442191717 Test RE 0.6777544256921281
51 Train Loss 4.915601 Test MSE 1.9770938455645453 Test RE 0.6720811307477002
52 Train Loss 4.8271217 Test MSE 2.0133483911379093 Test RE 0.6782152115882457
53 Train Loss 4.7976437 Test MSE 2.018289953204276 Test RE 0.6790470071810754
54 Train Loss 4.7386546 Test MSE 2.00053991600882 Test RE 0.6760544422780096
55 Train Loss 4.7155366 Test MSE 2.000428864293232 Test RE 0.676

40 Train Loss 1.6754146 Test MSE 1.3904840444400872 Test RE 0.5636260601084591
41 Train Loss 1.5390922 Test MSE 1.2372743152873984 Test RE 0.5316686576394668
42 Train Loss 1.3041542 Test MSE 1.2801106199874839 Test RE 0.5407939327722024
43 Train Loss 1.1753979 Test MSE 1.148128856553411 Test RE 0.5121573108690513
44 Train Loss 1.0633832 Test MSE 0.8459794711416164 Test RE 0.43963065982314037
45 Train Loss 0.83713746 Test MSE 0.45976411851950605 Test RE 0.3240974377007515
46 Train Loss 0.6013392 Test MSE 0.2986111085646921 Test RE 0.2611927032294166
47 Train Loss 0.4869387 Test MSE 0.1852959970299803 Test RE 0.2057505851140642
48 Train Loss 0.35941204 Test MSE 0.10074260777122711 Test RE 0.1517101558769071
49 Train Loss 0.32497147 Test MSE 0.09599857351787498 Test RE 0.1480950184393005
50 Train Loss 0.2994066 Test MSE 0.06506232209186914 Test RE 0.12191940651866778
51 Train Loss 0.28032768 Test MSE 0.057043379437998754 Test RE 0.11415913873097043
52 Train Loss 0.242225 Test MSE 0.044603

In [13]:
nan_tune

NameError: name 'nan_tune' is not defined

In [ ]:
import scipy.io as sio
import numpy as np

In [ ]:
for tune_reps in range(20):
    label = "KG_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

In [14]:
a = 0
for i in range(10):
    a = a+ test_re_full[i][-1]
    print(test_re_full[i][-1])
    
print("a = ",a/10)

0.013187398857835885
0.7916874700548301
0.015533489608380632
0.8403604832490177
0.009705293188510641
0.0145127096353756
0.8701002331425253
0.007181597648319921
0.6818812920230212
0.0396229497906773
a =  0.3283772917198494
